<a href="https://colab.research.google.com/github/Janith123gihan/Car-Damage-Classification/blob/main/Car_Damage_Classification_Transfer_Learning_EfficientNetB0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras import layers
import tensorflow as tf
from matplotlib import pyplot as plt
import os
import cv2
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from keras.utils import plot_model

In [ ]:

print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

2.9.2
Found GPU at: /device:GPU:0


In [ ]:
def set_data(train,test, validation ,batchSize, image_size):
 
 Image_size = [image_size,image_size]

 train_datagen= ImageDataGenerator(rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,rescale=1./255,
                                   zoom_range=0.2,
                                   horizontal_flip=0.5, vertical_flip=0.5
                                   )

 test_datagen = ImageDataGenerator(rescale=1./255)

 val_datagen = ImageDataGenerator(rescale=1./255)

 #Training dataset
 train_set = train_datagen.flow_from_directory(
                train,
                target_size=Image_size,
                batch_size=batchSize, 
                color_mode="rgb",
                #shuffle=True,             
                interpolation='bicubic',
                class_mode='categorical'
                )
 #Testing dataset
 test_set= test_datagen.flow_from_directory(
              test,
              target_size=Image_size,
              color_mode = "rgb", interpolation='bicubic',
              class_mode='categorical', shuffle=False
             )
 validation_set = val_datagen.flow_from_directory(
    validation, # same directory as training data
    target_size=Image_size,
    color_mode = "rgb",interpolation='bicubic',
    batch_size=batchSize)
 return train_set, test_set, validation_set;

In [ ]:
def unfreeze_model(model, num_of_layers):
    for layer in model.layers[num_of_layers:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True
    return model

In [ ]:
from keras.layers.pooling.global_average_pooling2d import GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, EfficientNetB1, Xception, EfficientNetB0
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model





basemodel = EfficientNetB0(include_top=False, input_shape=(128,128,3), weights="imagenet")
# basemodel.summary()
basemodel.trainable = False
basemodel = unfreeze_model(basemodel, -16)
for i, layer in enumerate(basemodel.layers):
    print(i, layer.name, layer.trainable)
x = basemodel.output
x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(5, activation="softmax", name="pred")(x)
model = tf.keras.Model(basemodel.input, outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


# base_model = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape = (224,224,3)))  #classification part is removed thatswhy include_top become false

# head_model = Model(base_model.input, base_model.layers[-2].output) #last two layers are removed
# head_model.trainable = False
# head_model = unfreeze_model(head_model,-3)  #last three layers are unfreezed

# x = head_model.output
# se = layers.GlobalAveragePooling2D(name="ch_pool")(x)
# se = layers.Reshape((1,1,512))(se)
# se = layers.Dense(32,activation="swish",kernel_initializer='he_normal', use_bias=False)(se)
# se = layers.Dense(512,activation="sigmoid",kernel_initializer='he_normal', use_bias=False)(se)
# x = layers.Multiply()([se, x])
# x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
# x = layers.BatchNormalization()(x) #it will normalize values to one - zero
# x = layers.Dropout(0.4)(x)
# x = layers.Dense(256, activation="relu",kernel_initializer="orthogonal")(x)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(128, activation="relu")(x)
# x = layers.Dropout(0.3)(x)


# outputs = layers.Dense(3, activation="softmax", name="pred")(x)
# model = tf.keras.Model(head_model.input, outputs) #model type is created
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
# model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["categorical_accuracy"])
# model.summary()
plot_model(model,show_shapes=True, show_layer_names=True)

In [ ]:
def plot_hist(hist):
    plt.figure(3)
    plt.plot(hist.history["categorical_accuracy"])
    plt.plot(hist.history["val_categorical_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
    
    plt.figure(4)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler

batchSize = 32
epoches =80;
image_size = 224;
train_path = '/content/drive/MyDrive/Car Damage Classification/training'
test_path = '/content/drive/MyDrive/Car Damage Classification/testing'
validation_path = '/content/drive/MyDrive/Car Damage Classification/validation'

# checkpoint_path = "/content/drive/MyDrive/Car Damage Classification/vgg16/checkpoint.hdf5"
# checkpoint_dir = os.path.dirname(checkpoint_path)
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                   monitor='val_categorical_accuracy',mode='max',
#                                                 save_best_only=True,
#                                                 verbose=1)



train_set, test_set, validation_set = set_data(train_path,test_path,validation_path, batchSize, image_size)

history=model.fit(train_set,  validation_data= validation_set,epochs = epoches, shuffle=True)
plot_hist(history)


In [ ]:
model.save('CNN-Car_Damage_Classification.h5')

# Testing Model

In [ ]:

model.save('/content/drive/MyDrive/Car Damage Classification/my_model')
results = model.evaluate(test_set,batch_size=32)
accuracy = results[1]
predict_labels=model.predict(test_set,batch_size=batchSize)
test_labels=test_set.classes
print(accuracy)


In [ ]:
print(test_labels)
print(predict_labels.argmax(axis=1))
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(test_labels, predict_labels.argmax(axis=1), target_names=['minor', 'moderate', 'severe']))
confusion = confusion_matrix(test_labels, predict_labels.argmax(axis=1))
print('Confusion Matrix\n')
print(confusion)
